The goal of this notebook is to take the ad hoc implementation in `MIPExample1.ipynb` and vectorize the operations such that any size Tyche problem can be converted to a MIP and solved.

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath("../src"))

In [3]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

from copy            import deepcopy
from IPython.display import Image 

In [4]:
import cProfile
import timeit

In [5]:
from mip import Model, MINIMIZE, MAXIMIZE, BINARY, xsum
from itertools import product

In [6]:
designs = ty.Designs("data")
investments = ty.Investments("data")
designs.compile()
tranche_results = investments.evaluate_tranches(designs, sample_count=250)
results = investments.tranches.join(tranche_results.summary)
evaluator = ty.Evaluator(investments.tranches, tranche_results.summary)

Get the wide-format interpolated elicitation data from the Tyche Evaluator and reset the multi-level index.

In [7]:
wide = evaluator.evaluate_corners_wide().reset_index()

In [8]:
wide.head(9)

Index,CIGS,CdTe,GaAs,InGaP,Perovskite,Polysilicon,Power Electronics,Soft Costs,Capital,Efficiency,GHG,Hazardous,LCOE,Lifetime,Strategic,Yield
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.533367,2.068183,-0.003592,0.970873,-0.142394,188.038770,0.06331,10006.391847
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,-1.460104,2.066785,-0.003592,0.972178,-0.140102,188.038727,0.06331,10006.399569
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,-1.297255,2.066043,-0.003592,0.973011,-0.135007,188.038694,0.06331,10006.394123
3,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,0.0,-1.471028,2.066579,-0.003592,0.960608,-0.136723,188.038751,0.06331,10070.686287
4,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1000000.0,-1.397766,2.065181,-0.003592,0.961913,-0.134431,188.038709,0.06331,10070.694009
5,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,5000000.0,-1.234917,2.064440,-0.003592,0.962746,-0.129336,188.038676,0.06331,10070.688563
6,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,0.0,-1.385462,2.067189,-0.003592,0.952804,-0.131992,188.038723,0.06331,10113.026230
7,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,1000000.0,-1.312200,2.065791,-0.003592,0.954109,-0.129699,188.038681,0.06331,10113.033952
8,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,5000000.0,-1.149351,2.065049,-0.003592,0.954942,-0.124605,188.038648,0.06331,10113.028506


**Input to MIP constructor needed**: List of investment categories

**Input to MIP constructor needed**: List of metrics to optimize and/or constrain

**Data check**: Confirm that all elements of both input lists match columns in the `evaluate_corners_wide()` data frame.

In [9]:
categories = ['Polysilicon', 'Power Electronics', 'Soft Costs']

metrics = ['Capital']

Pull out the investment values and metric values from the data set.

In [10]:
# Investment levels
inv_levels = wide.loc[:26,categories].values.tolist()

# Elicited metric values
m = wide.loc[:26,metrics].values.tolist()

# Number of endpoints
I = len(inv_levels)

Instantiate the MIP optimization problem.

In [11]:
example = Model(sense=MAXIMIZE)

Create binary (integer) variables $y_{in_i}$.

In [12]:
bin_vars = []

for i in range(I-1):
    _name = 'y' + '_' + str(i)
    bin_vars += [example.add_var(name=_name, var_type=BINARY)]
    del _name

Create continuous $\lambda$ variables with lower bound 0.0 and upper bound 1.0

In [13]:
lmbd_vars = []

for i in range(I):
    _name = 'lmbd' + '_' + str(i)
    lmbd_vars += [example.add_var(name=_name, lb=0.0, ub=1.0)]
    del _name

Define upper bound for total budget constraint and per-category upper bounds.

In [14]:
B = 3000000.0
max_amount = [1000000.0, 2000000.0, 1500000.0]

Create total budget constraint as a function of the $\lambda$ variables and the elicited investment levels from the data set.

In [15]:
example += xsum(lmbd_vars[i] * inv_levels[i][j] 
                for i in range(I)
                for j in range(len(inv_levels[i]))) <= B, 'Total Budget'

Create constraints for budget within each investment category.

In [16]:
for j in range(len(categories)):
    example += xsum(lmbd_vars[i] * [el[j] for el in inv_levels][i] 
                    for i in range(I)) <= max_amount[j], 'Budget for ' + categories[j]

Create three sets of constraints:
* Convexity constraints on $\lambda$ variables: the $\lambda$s must sum to 1.
* Constrain binary $y$ variables within each category such that the decision variables lie within exactly one interval (also a sum-to-1 constraint).
* Interval constraints on $y$ variables and $\lambda$ variables. There will be as many interval constraints per investment category as there are linear intervals.

In [17]:
example += sum(lmbd_vars) == 1, 'Lambda Sum'
example += sum(bin_vars) == 1, 'Binary Sum'

In [18]:
for i in range(I-1):
    example += bin_vars[i] <= lmbd_vars[i] + lmbd_vars[i+1], 'Interval Constraint' + str(i)

**In progress/Not implemented from this point onward**

Create objective function: Capital (metric) as a function of $\lambda$s and $y$s.

Create a data frame of $\lambda$ variables by mapping every element in `v` to its corresponding $\lambda$, using column name (investment category) and investment level.

In [19]:
example.objective = xsum(m[i][0] * lmbd_vars[i] for i in range(I))

Save model to a `lp` file for manual examination.

In [20]:
example.write('example.lp')

Optimize.

In [21]:
example.optimize()

<OptimizationStatus.OPTIMAL: 0>

Print optimal objective function value, with a negative applied to reverse the -1.0 in the objective function definition (minimizing a negative -> maximize a positive).

In [22]:
example.objective_value

-1.377409806638312

Get the optimal $\lambda$ and $y$ values.

In [27]:
lmbd_opt = []
y_opt = []

for v in example.vars:
    if 'lmbd' in v.name:
        lmbd_opt += [v.x]
    elif 'y' in v.name:
        y_opt += [v.x]

**Optional** Print the optimal variable values.

In [ ]:
print('{} : {}'.format(v.name, v.x))

Calculate the optimal investment levels.

In [26]:
for i in range(len(categories)):
    print(categories[i], 
          sum([lmbd_opt[j] * [el[i] for el in inv_levels][j] for j in range(len(lmbd_opt))]))

Polysilicon 2.96002454887102e-10
Power Electronics 1000000.0000000003
Soft Costs 1500000.0000000002
